# Creating & Uploading python distribution
https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container

In [ ]:
!python setup.py sdist

In [ ]:
#!gcloud storage cp dist/dat_package-0.1.tar.gz gs://dat-package-bucket
!gsutil cp dist/dat_package-0.1.tar.gz gs://dat-project-bucket/dat-package/

# Creating CustomJob
https://cloud.google.com/vertex-ai/docs/training/create-custom-job#create_custom_job-gcloud

# Operationalize Distributed Training with PyTorch on Google Cloud
https://www.youtube.com/watch?v=kXbDEmn2XOw


Pre-built-containers:
https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch

GPUs:
https://cloud.google.com/compute/docs/gpus
https://cloud.google.com/compute/gpus-pricing
https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table

In [ ]:
#!pip install google-cloud-aiplatform

In [27]:
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types import job_state
from google.cloud.aiplatform_v1 import types
from google.protobuf import json_format

project_id = 'sdml-dat'
region = 'us-central1' #TODO: change to better region
bucket_name = 'dat-project-bucket'
package_uri = f'gs://{bucket_name}/dat-package/dat_package-0.1.tar.gz'

# Initialize Vertex AI SDK
aiplatform.init(project=project_id, location=region, staging_bucket=bucket_name)

# Define worker pool spec
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,  # Number of masters
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "dat_package",
            "args": [],
        },
    }
    # ,
    # {
    #     "machine_spec": {
    #         "machine_type": "n1-standard-4",
    #         "accelerator_type": "NVIDIA_TESLA_K80",
    #         "accelerator_count": 1,
    #     },
    #     "replica_count": 2,  # Number of workers
    #     "python_package_spec": {
    #         "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
    #         "package_uris": [package_uri],
    #         "python_module": "dat_package",
    #         "args": [],
    #     },
    # },
]

# Define job name
job_name = 'dat-custom-pytorch-job'

# Create a custom job
job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)

# Launch the job
job.run(sync=True)

print(f"Job {job_name} has been launched.")

Creating CustomJob
CustomJob created. Resource name: projects/65830871515/locations/us-central1/customJobs/8410864692326563840
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/65830871515/locations/us-central1/customJobs/8410864692326563840')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8410864692326563840?project=65830871515
CustomJob projects/65830871515/locations/us-central1/customJobs/8410864692326563840 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8410864692326563840 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8410864692326563840 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8410864692326563840 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/841086469232656